# global vars

In [346]:
strTypD = {
    "root": "root",
    "stem": "stem",
    "lemma": "lem",
    "english": "eng",
    "All": "All"
}

strTypL = list(strTypD.keys())

frmL = ["All","i","ii","iii","iv","v","vi","vii","viii","ix","x","xi","xii",]

poSpD = {
    'All': 'All',
    'Adjective': 'ADJ',
    'Noun': 'N',
    'Proper noun': 'PN',
    'Verb': 'V',
    'Imperative verbal noun': 'IMPN',
    'Personal pronoun': 'PRON',
    'Demonstrative pronoun': 'DEM',
}

poSpL = list(poSpD.keys())

lngD = {
    "arabic": "arb",
    "english": "eng",
    "bengali": "bng"
}

lngL = list(lngD.keys())

lng2InpSchD = {
    "arabic": [
        "buckwalterScheme",
        "arabic_Scheme"
    ],
    "bengali": [
        "bengali_Scheme"
    ],
    "english": [
        "english_Scheme"
    ]
}
inpLngSchD = {
    "buckwalterScheme": "bkwSch",
    "arabic_Scheme": "arbSch",
    "bengali_Scheme": "bngSch",
    "english_Scheme": "engSch",
}

# root transliteration

In [347]:
def rtTrns(rt,inpLng,inpLngSch):
    lngSts = {
        "arb": "arbSch",
        "eng": "engSch",
        "bng": "bngSch"
    }

    bkwSch2arbSch = {
        "'": "ء",
        ">": "أ",
        "&": "ؤ",
        "<": "إ",
        "}": "ئ",
        "A": "ا",
        "b": "ب",
        "p": "ة",
        "t": "ت",
        "v": "ث",
        "j": "ج",
        "H": "ح",
        "x": "خ",
        "d": "د",
        "*": "ذ",
        "r": "ر",
        "z": "ز",
        "s": "س",
        "$": "ش",
        "S": "ص",
        "D": "ض",
        "T": "ط",
        "Z": "ظ",
        "E": "ع",
        "g": "غ",
        "_": "ـ",
        "f": "ف",
        "q": "ق",
        "k": "ك",
        "l": "ل",
        "m": "م",
        "n": "ن",
        "h": "ه",
        "w": "و",
        "Y": "ى",
        "y": "ي",
        "F": "ي",
        "N": "ٌ",
        "K": "ٍ",
        "a": "َ",
        "u": "ُ",
        "i": "ِ",
        "~": "ّ",
        "o": "ْ",
        "^": "ٓ",
        "#": "ٔ",
        "`": "ٰ",
        "{": "ٱ",
        ":": "ۜ",
        "@": "۟",
        '"': "۠",
        "[": "ۢ",
        ";": "ۣ",
        ",": "ۥ",
        ".": "ۦ",
        "!": "ۨ",
        "-": "۪",
        "+": "۫",
        "%": "۬",
        "]": "ۭ",
    }
    iasSch2arbSch = {

    }

    chrOut = {
        "arb": {
            "bkwSch": bkwSch2arbSch,
            "iasSch": iasSch2arbSch,
        },
        "eng": {
            "engSch": None
        },
        "bng": {
            "bngSc": None
        },
    }

    chrTrnsTbl = chrOut[inpLng][inpLngSch]
    rtTrns = ''
    if chrTrnsTbl != None:
        for chr in rt:
            chrTrns = chrTrnsTbl[chr]
            rtTrns += chrTrns
            # print("language schema is present\ncharacter transform: ", chrTrns)
    else:
        rtTrns = rt
    
    return rtTrns

# filechecker

In [348]:
def filecheck(filepath):
    import os
    BASE_DIR = os.getcwd()
    dirc = os.path.join(BASE_DIR,filepath)
    subdircL = filepath.split(os.sep)
    try:
        subdircL.remove('')
    except:
        pass
    doesPathExist = ''
    try:
        for supDirc in subdircL[:-1]:
            doesPathExist = os.path.join(doesPathExist,supDirc)
            if not os.path.exists(os.path.join(BASE_DIR,doesPathExist)):
                os.makedirs(os.path.join(BASE_DIR,doesPathExist))
                if os.path.exists(doesPathExist):
                    pass
        return os.path.isfile(os.path.join(BASE_DIR,filepath))
    except:
        pass

# table up

In [349]:
def tblUp(tblCumul,tblAgg,instDct,stri,lnk
            #   frm='',ptSp=''
                ):
        # for tbl in tbls:
        #     # print(tbl)
        #     tblAgg += tbl

        # print(len(tblAgg))
        
        print(f"number of instances of {stri} in {lnk} before Adam filter: {len(tblAgg)}")

        if len(tblAgg) > 0:
        # if len(tbls) > 0:
            # print(f"1st row of Table aggregate for {lnk} for {rt} is: {tblAgg[0].find_all('td')}")
            print(tblAgg[0].find_all('td')[1])
            if 'adam' in tblAgg[0].find_all('td')[1].get_text().lower():
            # if 'adam' in tbls[0].find_all('td')[1].get_text().lower():
                if stri.lower() != 'adam' and stri != 'A^dam':
                    # print("no results")
                    tblAgg = []
                    # tbls = []

        print(f"number of instances of {stri} in {lnk} after Adam filter: {len(tblAgg)}")
        # print(tblAgg)

        tblCumul += tblAgg
        # tblCumul += tbls

        print("length of tblCumul: ", len(tblCumul))
        # print(f"total number of instances so far of {rt} without removing duplicates: {len(tblCumul)}")

        for rw in tblCumul:
            row = [ fld.get_text() for fld in rw.find_all("td") ]
            # row = []
            # for fld in rw:
            #   row.append(fld.get_text())
        
            # print(row)
            pos = row[0].split(' ')[0].strip('()')
            # print(pos)
            if pos not in instDct:
                # print(posSplit)
                posSplit = pos.split(':')
                # print(posSplit)
                # instLst.append({
                instDct[pos] = {
                        "surah:ayah": f'{posSplit[0]}:{posSplit[1]}',
                        #   "position": int(posSplit[2]), 
                        "position": posSplit[2], 
                        "string": row[0].split(' ')[1], 
                        "meaning": row[1],
                        # "form": frm,
                        # "p-o-s": ptSp,
                        "ayah_link": row[2]
                    }
                # })
        
                # poss.add(pos)
            else:
                # if instDct[pos]["form"] == '' and frm != '':
                #     instDct[pos]["form"] = frm
                # if instDct[pos]["p-o-s"] == '' and ptSp != '':
                #     instDct[pos]["p-o-s"] = ptSp
                pass

            # print(f"number of unique instances upto {lnk}: {len(poss)} or {len(instLst)}")
            
        print(len(tblCumul), len(instDct))
        # print(tblAgg)

        return tblCumul, instDct


# filegetter

In [350]:
def fileGet(stri,filepath):
        import csv
        print(f"file found for {stri}")
        instDct = {}
        try:
            with open(filepath) as f:
                # print(f"loading {filepath}")
                instTbl = csv.DictReader(f, delimiter='\t')
                # instLst = [row for row in csv.DictReader(f, delimiter='\t') ]
                for row in instTbl:
                    surAyPos = f'{row["surah:ayah"]}:{row["position"]}'

                    instDct[surAyPos] = row

                print(f"Successfully loaded data from '{filepath}: lenght {len(instDct)}'")
                # propDat = True
        except:
            # propDat = False
            instDct = {}
            print(f"couldn't load data from '{filepath}': lenght {len(instDct)}")
        
        return instDct

# webGet

In [351]:
def webGet(stri,lnks,filepath,poSp,frm):
    # import html2text
    # import json
    import requests
    from bs4 import BeautifulSoup
    import re

    def getTbl(grabhtmlPara):
        soup = BeautifulSoup(
            grabhtmlPara,
            'lxml' 
        )

        tblsRet = soup.find_all(
            "table",
            {"class":"taf"}
        )

        return tblsRet


    # poss = set()
    instDct={}
    tblCumul = []
    # tblAgg = []
    print(f"proper data not found for {stri}")


    for lnk in lnks:
        print('\ngetting for link: ', lnk, '\n')
        if lnk == f'https://corpus.quran.com/qurandictionary.jsp?q={stri}':
            grabhtml = requests.get(lnk).text
            # print(type(grabhtml))

            import re

            headTbls = re.findall('(<h4 class="dxe">.*?(?=<h4))',grabhtml,re.DOTALL)

            # print(len(headTbls))

            for headTbl in headTbls:
                tblAgg = []
                soup = BeautifulSoup(
                    headTbl,
                    'lxml',
                    # 'html.parser',
                )
                head4 = soup.find_all(
                    "h4",
                    {"class":"dxe"}
                )[0]

                # if len(tblsRet) > 0:
                    # print(len(tblsRet) > 0)
                    # print(len(tblsRet))
                    # if len(tblsRet) == len(head4s):
                    # print(len(tblsRet) == len(head4s))
                # for i in range(len(head4s)):
                    # print(head4s[i].text)

                tblGrm = head4.text.split('-')[0]
                tblForms = re.findall('\(form (.*?)\)', tblGrm)
                if len(tblForms) == 0:
                    tblForm = 'All'
                else:
                    # print(tempForms[0])
                    tblForm = tblForms[0]
                tblPoSps = re.findall(f'(^[^\(\)]*?(?=\s*$|\s*\())', tblGrm)
                if len(tblPoSps) == 0:
                    tblPoSp = 'All'
                else:
                    # print(tempPtSps[0])
                    tblPoSp = tblPoSps[0]

                if tblForm == 'All':
                    if tblPoSp != 'All':
                        tblForm = 'I'
                print(
                    # grm, 
                    tblForm, 
                    tblPoSp
                )

                tbls = soup.find_all(
                    "table",
                    {"class":"taf"}
                )

                # tblAgg = [ tbl for tbl in tbls ]

                # print(tblsRet)
                for tbl in tbls:
                    if tblPoSp == poSp and tblForm == frm:
                        tblAgg += tbl
                print("tblAgg length", len(tblAgg))
                tblCumul,instDct = tblUp(tblCumul,tblAgg,instDct,
                                        #  tempForm,tempPtSp
                                        stri,lnk
                                            )


        else:
            pgs = []
            tblAgg = []
            grabhtml = requests.get(f"{lnk}").text
            tbls = getTbl(grabhtml)
            if 'Results' in grabhtml:
                matches = re.findall(">[\n\s]*Results[\s\n]*<b>\d*</b>[\s\n]*to[\s\n]*<b>\d*</b>[\s\n]*of[\s\n]*<b>(\d*)</b>", grabhtml, re.DOTALL)
                print('\nmatches: ', matches)
                if len(matches) > 0:
                    pgFlt = int(matches[0])/50
                    pgCount = int(pgFlt) + 1 if int(matches[0]) % 50 != 0 else int(pgFlt)
                    # print(f"page count in {lnk} is: {pgCount}")
                    # print(type(pgCount), pgCount)
                    pgs = list(map(lambda x : f'&page={x}', list(range(2,pgCount+1))))

                for pg in pgs:
                    print(f'\nin pg {pg}')
                    grabhtml = requests.get(f"{lnk}{pg}").text
                    tbls += getTbl(grabhtml)
                    # print(f"length of grabhtml is: {len(grabhtmlNew)}")
                    # grabhtml += grabhtmlNew
                    # print(f"length of grabhtml after adding is: {len(grabhtml)}")
                
            for tbl in tbls:
                tblAgg += tbl
                
            print("length of tblAgg: ", len(tblAgg))
            
            tblCumul,instDct = tblUp(tblCumul,tblAgg,instDct,stri,lnk)

        # print(f"\nnumber of tables found in {lnk} for {rt} is: {len(tbls)}")
        # print(f"{tbls}\n")


    list_header = ['surah:ayah', 'position', 'string', 'meaning', 'form', 'p-o-s', 'ayah_link']
    print(f"writing {stri} to '{filepath}'")
    with open(f'{filepath}', 'x') as f:
        # print(f"writing {stri} to '{filepath}'")
        import csv
        writer = csv.DictWriter(f, delimiter='\t', fieldnames=list_header)
        writer.writeheader()
        # for datum in instLst:
        for k, datum in instDct.items():
            writer.writerow({
                list_header[0] : datum['surah:ayah'],
                list_header[1] : datum['position'],
                list_header[2] : datum['string'],
                list_header[3] : datum['meaning'],
                # list_header[4] : datum['form'],
                # list_header[5] : datum['p-o-s'],
                list_header[6] : datum['ayah_link'],
            })
    
    return instDct

# datagrabber

In [352]:
def dataGrabber(
        strObj
    ):

    '''Grabs data from corpus.quran.com & formats them'''

    flt = str(strObj["flt"]).lower()
    strTyp = strTypD[strObj["strTyp"]]
    frm = strObj["frm"]
    poSp = poSpD[strObj["poSp"]]
    inpLng = lngD[strObj["inpLng"]]
    inpSch = inpLngSchD[strObj["inpSch"]]
    stri = strObj["stri"]
    filename = rtTrns(strObj["stri"],inpLng,inpSch)


    if stri == "" or strTyp == "eng" "aeiou" not in stri and strTyp == "All":
        strTyp = "root"

    mainLnk = "https://corpus.quran.com/search.jsp?q="
    isArabic = strTyp != 'eng' and stri !='' and inpLng == 'arb'

    if not isArabic:
        if flt != '' and stri =='':
            stri = str(flt).lower()
            flt = ''
    lnks = []
    if isArabic:
        filepath = f'data/cache/arb/{filename}/{strTyp}/{frm}/{poSp}.tsv'
        if poSp != "All":
            mainLnk += f"pos:{stri} "
        if frm != "All":
            mainLnk += f"({frm}) "
        if strTyp !='All':
            mainLnk += f"{strTyp}:"
        if strTyp == 'All':
            for strTy in ['lem','stem','room']:
                lnks.append(
                    f"https://corpus.quran.com/search.jsp?q={strTy}:{stri}"
                )
            # lnks = [
            #     f"https://corpus.quran.com/search.jsp?q=lem:{stri}",
            #     f"https://corpus.quran.com/search.jsp?q=stem:{stri}",
            #     f"https://corpus.quran.com/search.jsp?q=root:{stri}",
            # ]
        # lnks = [mainLnk]
        mainLnk += stri
        lnks = [*lnks, f"https://corpus.quran.com/qurandictionary.jsp?q={stri}"]
    else:
        # lnks = [mainLnk]
        filepath = f'data/cache/nonarb/{stri}.tsv'
        mainLnk += stri
    lnks = [*lnks,mainLnk]
    
    # instLst=[]

    fileFound = filecheck(filepath)
    # if len(links) != 1:
    # import os
    # for k, direcLnk in direcLnkDic.items():
        # propDat, 
    instDct = fileGet(stri,filepath) if fileFound else webGet(stri,lnks,filepath,poSp,frm,)
        # for lnk in lnks:

    # posDic = {
    #     'ADJ': 'Adjective',
    #     'N': 'Noun',
    #     'PN': 'Proper noun',
    #     'V': 'Verb',
    #     'IMPN': 'Imperative verbal noun',
    #     'PRON': 'Personal pronoun',
    #     'DEM': 'Demonstrative pronoun',
    #     '': '',
    # }
    
    # print(frmReq, ptSpReq)
        # print(len(instLst))
    instLst = [datum for datum in instDct.values()]
    # if frmReq != None:
    #     # instLst = [datum if datum["form"] == frmReq.capitalize() else None for datum in instLst]
    #     instLst = list(filter(lambda datum: datum["form"] == frmReq.capitalize(), instLst))
    # if ptSpReq != None:
    #     # instLst = [datum if datum["p-o-s"] == posDic[ptSpReq] else for datum in instLst]
    #     instLst = list(filter(lambda datum: datum["p-o-s"] == posDic[ptSpReq], instLst))
        

    return instLst

# filter

In [353]:
def filtDown(strObj):
    stri = strObj["stri"]
    flt = strObj["flt"]
    import re
    instLst = dataGrabber(strObj)
    instLstFiltered =  list(filter( lambda row : len(re.compile(str(flt).lower()).findall(row["meaning"].lower())) > 0 or len(re.compile(str(stri)).findall(row["meaning"])), instLst))
    # instLstFiltered = [ { **row , "query" : f"{rt} ({flt})"} for row in instLstFiltered ]
    return instLstFiltered

# intsct

In [368]:
{1,2,3}.intersection({})

set()

In [381]:
def intersct(comb):
    strL = comb["strL"]
    if len(strL) > 0:
        instLstAgg = []
        surahAyahAggSet = set()
        for i in range(len(strL)):
            strObj = strL[i]
            instLst = filtDown(strObj)
            surahAyahList = [ inst["surah:ayah"] for inst in instLst ]
            if i == 0:
            # if surahAyahAggSet == set([]):
                surahAyahAggSet = set(surahAyahList)
            else:
                surahAyahAggSet = surahAyahAggSet.intersection(surahAyahList)
            instLstAgg += instLst
        
        instLstFlt = list(filter(
            lambda x: x["surah:ayah"] in surahAyahAggSet,
            instLstAgg
        ))
        instDictInc = {}
        for inst in instLstFlt:
            surahAyah = inst["surah:ayah"]
            if surahAyah not in instDictInc.keys():
                instDictInc[surahAyah] = {
                    "position": inst["position"],
                    "string": inst["string"],
                    "meaning": inst["meaning"],
                    "ayah_link" : inst["ayah_link"],
                    # "query": 
                    # f"{rtAgg} ({flAgg})"
                    # stri_flt_int_lb
                }
            else:
                instDictInc[surahAyah]["string"] = instDictInc[surahAyah]["string"] + ' ' + inst["string"]
                instDictInc[surahAyah]["meaning"] = instDictInc[surahAyah]["meaning"] + ' ' + inst["meaning"]

        instLstInc = [ {"surah:ayah":k, **v } for k, v in instDictInc.items() ]
        return instLstInc
    
    # elif len(strL) == 1:
    #     instLstFlt = filtDown(strL[0])
    #     instLstInc = [ { **rec, "query": f"{stri_flt_int_lb})" } for rec in instLstFlt ]
    #     return instLstInc
    
    else:
        print("please provide at least one root/word")

# aggreg

In [376]:
def aggregLsts(qL,tafs="ar-tafsir-al-tabari"):
    # lnkStyle = ' '
    fontSize = '18'
    fontCol = 'rgb(0,0,150)'
    shPx = str(int(fontSize)/100*0)
    shCol = '#000000'
    bgCol = 'rgb(220,220,250)'
    txtShad = ''
    # txtShad = f'text-shadow:{shPx}px {shPx}px 0 {shCol}, {shPx}px -{shPx}px 0 {shCol}, -{shPx}px -{shPx}px 0 {shCol}, -{shPx}px {shPx}px 0 {shCol};'
    # lnkStyle = "style='background-color:rgb(250,250,250);font-size:30px;color:rgb(0,0,150);-webkit-text-stroke-width:5px;-webkit-text-stroke-color:white' "
    lnkStyle = f"style='background-color:{bgCol};font-size:{fontSize}px;color:{fontCol};{txtShad}' "
    # lnkStyle = "style='color:rgb(250,250,250);-webkit-text-stroke-width:1px;-webkit-text-stroke-color:rgb(0,0,0);' "
    instLstAgg = []
    for optSt in qL:
        print(optSt)
        lbl = ''
        for comb in optSt:
            print(comb)
            strL = comb["strL"]
            lbl += ' / '.join([ 
                ' '.join([
                    strObj["stri"],
                    strObj["flt"]
                ]) for 
                strObj 
                in strL
            ])
            instLst = intersct(comb)
                
            instLst = [ { **inst, "query": lbl } for inst in instLst  ]
            instLstAgg += instLst
    instLstAgg = [ { 
        **row, 
        "ayah_link": f"<a {lnkStyle}href='https://quran.com/{row['surah:ayah']}/tafsirs/{tafs}'>{row['ayah_link']}</a>"
        # "ayah_link": f"<a href='https://quran.com/{row['surah:ayah']}/tafsirs/{tafs}'>{row['ayah_link']}</a>"
        } for row in instLstAgg ]
    print(f"\ntotal {len(instLstAgg)} instances found")
    return instLstAgg

# classes

In [377]:
class strObjClass:
    # idx = 0
    # parentIdx = 1
    striSt = ''
    fltSt=''
    strTypSt='root'
    frmSt='All'
    poSpSt='All'
    inpLngSt="arabic"
    inpSchSt="buckwalter_Scheme"


    def __init__(self,                
                stri = striSt,
                flt = fltSt,
                strTyp = strTypSt,
                frm = frmSt,
                poSp = poSpSt,
                inpLng = inpLngSt,
                inpSch = inpSchSt ,
                #  strSt=strObjStClass()
                #  **kwargs
                ):
        
        self.stri = stri
        self.flt = flt
        self.strTyp = strTyp
        self.frm = frm
        self.poSp = poSp
        self.inpLng = inpLng
        self.inpSch = inpSch
        self.strObj = {
            "stri": stri,
            "flt": flt,
            "strTyp": strTyp,
            "frm": frm,
            "poSp": poSp,
            "inpLng": inpLng,
            "inpSch": inpSch,
        }


In [357]:
class combClass:
    strLSt = []
    vrsLsSt = 0
    def __init__(self,strL=strLSt,vrsDis=vrsLsSt):
      # print(combsLsA)
      self.vrsDis = vrsDis
      self.strL = strL
      for strObj in strL:
          self.strL.append(strObjClass(strObj))

# widget

In [358]:
import ipywidgets as widg
from ipywidgets import interactive as intct
from IPython.display import display

In [359]:
class strObWdgCl:
    # global fulWdgD
    # idx = 0
    # parentIdx = 1
    strngs = []
    strTypSt = "root"
    frmSt = "All"
    poSpSt = "All"
    inpLngSt = "arabic"
    inpSchSt = "buckwalterScheme"

    def update_language_scheme_options(self,*args):
        # print(self.inpLngW.value)
        self.inpSchW.options=lng2InpSchD[self.inpLngW.value]
        self.inpSchW.value=lng2InpSchD[self.inpLngW.value][0]
    
    # def entStrObjM(self,*args):
    def entStrObjM(self,button,
                #    strngs,comb_container
                   ):
        strObWdgCl.strTypSt=self.strTypeW.value
        strObWdgCl.frmSt=self.frmW.value
        strObWdgCl.poSpSt=self.poSpW.value
        strObWdgCl.inpLngSt=self.inpLngW.value
        strObWdgCl.inpSchSt=self.inpSchW.value

        strObWdgCl(
            self.parentComb
                                 )

    def delStrObjM(self,button,
                   ):
        if len(self.parentComb.strngs) > 1:
            if self.strContainer in self.parentComb.strngs:
                self.parentComb.strngs.remove(self.strContainer)
                self.parentComb.comb_container.children = [w for w in self.parentComb.comb_container.children if w != self.strContainer]

    def __init__(self,
                 parentComb,
                ):

        self.parentComb = parentComb

        self.striW = widg.Text(description=f"String_Object")
        self.fltW = widg.Text(description=f"Translation_filter")
        self.strTypeW = widg.Dropdown(options=strTypL, value=self.strTypSt,description=f"String_type")
        self.poSpW = widg.Dropdown(options=poSpL, value=self.poSpSt,description=f"Part_of_speech")
        self.frmW = widg.Dropdown(options=frmL, value=self.frmSt,description=f"Form")
        self.inpLngW = widg.Dropdown(options=lngL, value=self.inpLngSt,description=f"Input_language")
        self.inpSchW = widg.Dropdown(description=f"Input_scheme")
        self.entStrObjB = widg.Button(description=f"Enter String Object")
        self.delStrObjB = widg.Button(description=f"Delete String Object")
        self.entStrObjB.on_click(self.entStrObjM)
        self.delStrObjB.on_click(self.delStrObjM)
        self.inpLngW.observe(self.update_language_scheme_options)
        self.update_language_scheme_options(self)

        self.strContainer = widg.VBox(
            [
                self.striW,
                self.fltW,
                self.strTypeW,
                self.poSpW,
                self.frmW,
                self.inpLngW,
                self.inpSchW,
                self.entStrObjB,
                self.delStrObjB
            ]
        )

        parentComb.strngs.append(self.strContainer)

        parentComb.comb_container.children = [parentComb.comb_container.children[0], self.strContainer, *parentComb.comb_container.children[1:]]

        parentComb.comb_container.layout=widg.Layout(
            max_width="800px",       # Set the width to control the horizontal space
            # max_height="800px",       # Set the width to control the horizontal space
            # width="500px",       # Set the width to control the horizontal space
            # height="200px",       # Set the width to control the horizontal space
            # overflow="scroll",  # Enable horizontal scrolling if content overflows
            border="1px solid black",  # Optional: add a border to make the scroll area visible
            # display="flex",
            # flex_flow='row',
            overflow='scroll hidden'
        )


In [360]:
class combWdgCl:
        # Function to add a new group of widgets
    def entCombM(self,button):
        # new_group_id = len(combs) + 1
        # new_group = 
        print(len(self.opt_container.children[1].children))
        combWdgCl(
            # new_group_id
            self.combs,
            self.opt_container
            )
        print(len(self.opt_container.children[1].children))
        
        # self.combs.append(new_group)
        # self.container.children = list(self.container.children) + [new_group]

    # Function to remove this specific group of widgets
    def delCombM(self,button):
        if len(self.combs) > 1:
            if self.comb_container in self.combs:
                self.combs.remove(self.comb_container)
                self.opt_container.children[1].children = [w for w in self.opt_container.children[1].children if w != self.comb_container]
    # Function to create a new group of widgets with its own Add and Remove buttons
    def __init__(self,
            # comb_id
                 combs,
                 opt_container
            ):
        # Widgets for the group (e.g., Text and Slider)

        self.combs = combs
        self.opt_container = opt_container
        # print(len(self.container.children))

        self.vrsDistW = widg.IntText(min=0,max=6236,value=0, description=f"Verse Distance")
        self.entCombB = widg.Button(description="Enter Combination of String Objects")
        self.entCombB.on_click(self.entCombM)
        self.delCombB = widg.Button(description="Delete Combination of String Objects")
        self.delCombB.on_click(self.delCombM)

        self.comb_container = widg.HBox(
            [
                widg.VBox(
                    [
                        self.vrsDistW,
                        widg.HBox([self.entCombB, self.delCombB]),
                    ],
                    # layout = widgets.Layout(
                    #     width="200px"
                    # )
                )
            ],
                # layout=widgets.Layout(
                #     width="500px",       # Set the width to control the horizontal space
                #     overflow_x="scroll",  # Enable horizontal scrolling if content overflows
                #     border="1px solid black"  # Optional: add a border to make the scroll area visible
                # )   
        )

        # strngs_container = widgets.HBox()

        self.strngs = []

        strObWdgCl(self)

        # strngs_container.children = [initial_strng]

        # strngs.append(initial_strng)

        self.combs.append(self.comb_container)
        self.opt_container.children[1].children = [self.comb_container,*self.opt_container.children[1].children,]



In [361]:
class optStWdgCl:
        # Function to add a new group of widgets
    def entOptStM(self,button):
        # new_group_id = len(combs) + 1
        # new_group = 
        print(len(self.container.children))
        optStWdgCl(
            # new_group_id
            self.optSts,
            self.container
            )
        print(len(self.container.children))
        

    # Function to remove this specific group of widgets
    def delOptStM(self,button):
        if len(self.combs) > 1:
            if self.optSt_container in self.optSts:
                self.optSts.remove(self.optSt_container)
                self.container.children = [w for w in self.container.children if w != self.optSt_container]
    # Function to create a new group of widgets with its own Add and Remove buttons
    def __init__(self,
            # comb_id
                 optSts,
                 container
            ):
        # Widgets for the group (e.g., Text and Slider)

        self.optSts = optSts
        self.container = container
        # print(len(self.container.children))

        # self.vrsDistW = widg.IntText(min=0,max=6236,value=0, description=f"Verse Distance")
        self.entOptStB = widg.Button(description="Enter Option of String Objects")
        self.entOptStB.on_click(self.entOptStM)
        self.delOptStB = widg.Button(description="Delete Option of String Objects")
        self.delOptStB.on_click(self.delOptStM)

        self.optSt_container = widg.VBox(
            [
                widg.HBox(
                    
                        [self.entOptStB, self.delOptStB],
                    
                    # layout = widgets.Layout(
                    #     width="200px"
                    # )
                ),
                widg.HBox([])
            ],
                # layout=widgets.Layout(
                #     width="500px",       # Set the width to control the horizontal space
                #     overflow_x="scroll",  # Enable horizontal scrolling if content overflows
                #     border="1px solid black"  # Optional: add a border to make the scroll area visible
                # )
        )

        # strngs_container = widgets.HBox()

        self.combs = []

        combWdgCl(
            self.combs,
            self.optSt_container
        )

        self.optSts.append(self.optSt_container)
        self.container.children = [self.optSt_container,*self.container.children,]



In [389]:
# Container to hold all groups of widgets
container = widg.VBox(
    # layout=widgets.Layout(
    #             width="600px",       # Set the width to control the horizontal space
    #             overflow_x="scroll",  # Enable horizontal scrolling if content overflows
    #             border="1px solid black"  # Optional: add a border to make the scroll area visible
    #         )   
)

# List to track widget groups
combs = []
# Initialize the first group of widgets
optStWdgCl(
    # 1
    combs,
    container
    )


In [ ]:
qL = []
def finish_query_f(button):
    for k in range(len(container.children)-1,-1,-1):
        optStC = container.children[k].children[1]
        optSt = []
        for l in range(len(optStC.children)-1,-1,-1):
            combC = optStC.children[l]
            if not len(combC.children) < 2:
                vrsDisFld = combC.children[0].children[0]
                print(vrsDisFld.description, vrsDisFld.value)
                # combObj = combClass()
                combObj = {"strL":[],"vrsDis":vrsDisFld.value}
                for i in range(len(combC.children)-1,0,-1):
                    strCs = combC.children[i]
                # for strObj in comb.children:
                    # print('\n', strCs)
                    strCFlds = strCs.children
                    # print(strCFlds)
                    strAtts = ["stri","flt","strTyp","poSp","frm","inpLng","inpSch"]
                    # if True:
                    # strD = strObjClass()
                    strD = {}
                    if strCFlds[0].value != '' or strCFlds[1].value != '':
                        for j in range(7):
                            fld = strCFlds[j]
                            # print(fld.description, fld.value)
                            # print(strD[strAtts[j]], fld.value)
                            # strD.strObj[strAtts[j]] = fld.value
                            strD[strAtts[j]] = fld.value
                        # print(strD)
                        # print(strD.strObj)
                        # combObj.strL.append(strD.strObj)
                        combObj["strL"].append(strD)
                # if len(combObj.strL) > 0:
                if len(combObj["strL"]) > 0:
                    optSt.append(combObj)
            if len(optSt) > 0:
                qL.append(optSt)
                # qL.append(combObj.__dict__)
    dg = aggregLsts(qL)
    # return dg
finish_query_B = widg.Button(description="Add Combination", layout=widg.Layout(width="auto"))
finish_query_B.on_click(finish_query_f)

Verse Distance 0
[{'strL': [{'stri': 'rbn', 'flt': '', 'strTyp': 'root', 'poSp': 'All', 'frm': 'All', 'inpLng': 'arabic', 'inpSch': 'buckwalterScheme'}, {'stri': 'khn', 'flt': '', 'strTyp': 'root', 'poSp': 'All', 'frm': 'All', 'inpLng': 'arabic', 'inpSch': 'buckwalterScheme'}], 'vrsDis': 0}]
{'strL': [{'stri': 'rbn', 'flt': '', 'strTyp': 'root', 'poSp': 'All', 'frm': 'All', 'inpLng': 'arabic', 'inpSch': 'buckwalterScheme'}, {'stri': 'khn', 'flt': '', 'strTyp': 'root', 'poSp': 'All', 'frm': 'All', 'inpLng': 'arabic', 'inpSch': 'buckwalterScheme'}], 'vrsDis': 0}
file found for rbn
Successfully loaded data from 'data/cache/arb/ربن/root/All/All.tsv: lenght 0'
file found for khn
Successfully loaded data from 'data/cache/arb/كهن/root/All/All.tsv: lenght 2'

total 0 instances found


In [391]:
display(container,finish_query_B)

Button(description='Add Combination', layout=Layout(width='auto'), style=ButtonStyle())